Imports

In [4]:
from Dataset import processData
from Model import train, KGLinkPredictor
import torch

Process Data

In [5]:
data, ptrain_loader, pval_loader,ftrain_loader, fval_loader, ftest_loader = processData(256,2048,'totalkg.csv') # type: ignore

Loading data from path: /opt/scratch/labs/wuc/Drug-Repurposing/data/totalkg.csv
Processing node data...
Processing edge data...
Setting up pretraining data...
Setting up finetuning data...
Processing disease similarities... (THIS WILL TAKE AROUND 40 MINUTES)


Declare and Train Model

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = KGLinkPredictor(256,64,data).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
data = data.to(device)

print('Pretraining...')
train(ptrain_loader,pval_loader, model, optimizer, device, 1) # Takes about 30 mins

model.Decoder.reset_parameters()

print('Fine-tuning...')
train(ftrain_loader,fval_loader, model, optimizer, device, 50)

Pretraining...


  0%|          | 0/1 [00:00<?, ?it/s]

IndexError: index 83874 is out of bounds for dimension 0 with size 65873